In [ ]:
#| hide


# earthengine-dask

> Scale up concurrent requests to Earth Engine interactive endpoints with Dask

This file will become your README and also the index of your documentation.

# Prerequisites

- A [Google Earth Engine](https://earthengine.google.com/) account.
- Access to a Google Cloud Platform (GCP) [project with the Earth Engine API enabled](https://developers.google.com/earth-engine/cloud/earthengine_cloud_project_setup).
- A [Coiled](https://www.coiled.io/) account that is [setup to use the GCP project](https://docs.coiled.io/user_guide/setup/gcp/cli.html).

## Install with PIP

```sh
pip install earthengine_dask
```

## How to use

## Import Python packages

In [ ]:
import altair as alt
import ee
from earthengine_dask.core import ClusterGEE
import google.auth
import pandas as pd

## Authenticate & Initialize Earth Engine

Get credentials and the GCP project ID, authenticating if necessary.

In [ ]:
try:
    credentials, project_id = google.auth.default()
except google.auth.exceptions.DefaultCredentialsError:
    !gcloud auth application-default login
    credentials, project_id = google.auth.default()

ee.Initialize(credentials=credentials, project=project_id)

# Start Dask Cluster

Start up a Earth Engine enabled cluster. This may take a few minutes to complete.

In [ ]:
cluster = ClusterGEE(
    name='test-class-cluster',
    n_workers=2,
    worker_cpu=8,
    region='us-central1',
)

Output()

╭──────────────────────────────── Package Info ────────────────────────────────╮
│                                 ╷                                            │
│   Package                       │ Note                                       │
│ ╶───────────────────────────────┼──────────────────────────────────────────╴ │
│   coiled_local_earthengine_dask │ Source wheel built from                    │
│                                 │ ~/Documents/GitHub/VorGeo/earthengine-da   │
│                                 │ sk                                         │
│                                 ╵                                            │
╰──────────────────────────────────────────────────────────────────────────────╯

Output()

Google Application Default Credentials have been written to a file on your Coiled VM(s).
These credentials will potentially be valid until explicitly revoked by running
gcloud auth application-default revoke


Retrieve a client for the cluster, and display it.

In [ ]:
client = cluster.get_client()
client

<Client: 'tls://10.2.0.16:8786' processes=2 threads=16, memory=61.18 GiB>

# Submit Jobs

Test it out by:
- Defining a function that can be distributed,
- Submitting jobs running the function to workers, 
- Gathering the results locally, and
- Displaying the results

In [ ]:
# Get a list of countries to analyze.
country_fc = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
country_list = country_fc.aggregate_array('country_na').distinct().sort().getInfo()

# Write a function that can be run by the cluster workers. 
def get_country_stats(country_name):
    country = country_fc.filter(ee.Filter.eq('country_na', country_name))
    elev = ee.ImageCollection("COPERNICUS/DEM/GLO30").select('DEM').mosaic()
    return {
        'country': country_name, 
        'area_km2': country.geometry().area().multiply(1e-6).round().getInfo(), 
        'mean_elev': elev.reduceRegion(reducer=ee.Reducer.mean(),
                                       geometry=country.geometry(),
                                       scale=10000,
                                       ).get('DEM').getInfo(),
    }

# Create and submit jobs to among the workers.
submitted_jobs = [
    client.submit(get_country_stats, country)
    for country in country_list
]

# Gather up the results and display them.
results = client.gather(submitted_jobs)
df = pd.DataFrame(results)
df

,country,area_km2,mean_elev
0,Abyei Area,10460,402.592190
1,Afghanistan,642093,1809.717311
2,Akrotiri,127,60.796081
3,Aksai Chin,30448,5324.949965
4,Albania,28638,689.740847
...,...,...,...
279,West Bank,5813,341.715816
280,Western Sahara,269689,253.175100
281,Yemen,454682,932.051351
282,Zambia,754032,1118.995950


Display a scatter plot of the data.

In [ ]:
alt.Chart(df).mark_circle(size=60).encode(
    x='area_km2',
    y='mean_elev',
    tooltip=['country']
).interactive()

alt.Chart(...)

# Shut down the cluster

In [ ]:
cluster.shutdown()